<a href="https://colab.research.google.com/github/Shibu4064/ML_LAB/blob/main/sagor_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate

In [ ]:
import pandas as pd
# train_df = pd.read_csv('/content/drive/MyDrive/EMNLP/train.csv') # train set
# val_df = pd.read_csv('/content/drive/MyDrive/EMNLP/dev.csv') # validation set
train_df=pd.read_csv('/content/drive/MyDrive/EMNLP/df12_cleaned.csv')

In [ ]:
print(len(train_df))

6531


In [ ]:
train_df.columns

Index(['Unnamed: 0', 'Text', 'label'], dtype='object')

In [ ]:
train_df.head()

,Unnamed: 0,Text,label
0,0,ইসলামের সচেতন সৈনিককুত্তার বাচ্চা তোদের শিক্ষা...,1.0
1,1,ধর্ম মানে অশান্তি,1.0
2,2,হিন্দু প্রকারভেদ রাধাকৃষ্ণ কৃষ্ণ_কৃষ্ণ বুঝলে...,1.0
3,3,সালা তুই একটা হিন্দুর বাচ্চা নামের পরে খান লা...,2.0
4,4,হিন্দুদের দেশ বাংলাদেশ জয় শ্রী রাম,1.0


In [ ]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
#val_texts, val_labels = pandaToList(val_df)

In [ ]:
!pip uninstall jaxlib

Found existing installation: jaxlib 0.4.13
Uninstalling jaxlib-0.4.13:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/jaxlib-0.4.13.dist-info/*
    /usr/local/lib/python3.10/dist-packages/jaxlib/*
Proceed (Y/n)? y
  Successfully uninstalled jaxlib-0.4.13


In [ ]:
pip install -U jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 8.7 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.14-py3-none-any.whl size=1535362 sha256=0fd04c3f7c538a72d8b45f04157e565d616d741422c5de2df93f024e956b6301
  Stored in directory: /root/.cache/pip/wheels/85/52/e7/dfa571c9f9b879e3facaa1584f52be04c4c3d1e14054ef40ab
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.4.13
    Uninstalling jax-0.4.13:
      Successfully uninstalled jax-0.4.13


In [ ]:
tokenizer_name = 'sagorsarker/bangla-bert-base'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=1e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

model_name = 'sagorsarker/bangla-bert-base'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) ## change here for using another pretrained model

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,1.099956,0.572932,0.439620,0.372984,0.314589
500,1.410400,1.308242,0.608271,0.427373,0.418581,0.384824
750,1.410400,1.217576,0.635338,0.657721,0.465038,0.448334
1000,1.310300,1.337116,0.663910,0.648499,0.595552,0.603167
1250,1.310300,1.542084,0.672932,0.629668,0.596018,0.608713
1500,1.411900,1.435159,0.690226,0.666188,0.580657,0.602401
1750,1.411900,1.487779,0.683459,0.682290,0.554824,0.575867
2000,1.598800,1.368622,0.678947,0.680870,0.571739,0.581075
2250,1.598800,1.482245,0.695489,0.719635,0.578384,0.608558
2500,1.320900,1.411795,0.700000,0.675560,0.604047,0.624200


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=9500, training_loss=0.9776337280273437, metrics={'train_runtime': 2077.3683, 'train_samples_per_second': 64.986, 'train_steps_per_second': 64.986, 'total_flos': 912931614441000.0, 'train_loss': 0.9776337280273437, 'epoch': 3.52})

In [ ]:
trainer.predict(val_dataset)

PredictionOutput(predictions=array([[ 4.773113 , -1.771134 , -3.158644 ],
       [ 5.023822 , -1.5623549, -3.8756378],
       [ 5.457322 , -1.9931476, -3.6866221],
       ...,
       [ 5.0701985, -1.65752  , -3.7112646],
       [-2.025107 ,  4.927438 , -3.3979213],
       [-0.6819402, -3.190272 ,  3.8797836]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 1.5676639080047607, 'test_accuracy': 0.7330827067669173, 'test_precision': 0.7051179575119383, 'test_recall': 0.6920230701272813, 'test_f1': 0.6974740325681054, 'test_runtime': 28.2753, 'test_samples_per_second': 47.038, 'test_steps_per_second': 47.038})

In [ ]:
output_dir = "/content/drive/MyDrive/VITD-Models/sagorbert"

In [ ]:
trainer.save_model(f"{output_dir}") # saving the model to drive.